<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-historical-daily-exchange-rates-ECB" data-toc-modified-id="Load-historical-daily-exchange-rates-ECB-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load historical daily exchange rates ECB</a></span></li><li><span><a href="#Seed-the-table" data-toc-modified-id="Seed-the-table-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Seed the table</a></span></li></ul></div>

## Load historical daily exchange rates ECB

In [39]:
# import wget
import numpy as np
import pandas as pd
import pandas_gbq
import logging

In [54]:
def load_data(opt:str='new'):
    """
    opt: {'hist', 'new'}
    """
    urls = {
        "hist":"https://www.ecb.europa.eu/stats/eurofxref/eurofxref-hist.zip",
        "new":"https://www.ecb.europa.eu/stats/eurofxref/eurofxref.zip"}
    hist= pd.read_csv(urls[opt], compression='zip')
    return hist

def process_data(hist:pd.DataFrame)->pd.DataFrame:
    value_cols = [x for x in hist.columns  if (x.strip() not in ['Date', ''])and('Unnamed' not in x)]
    hist = hist.pipe(pd.melt,
                    id_vars=['Date'], 
                    var_name='CurrencyCode',
                    value_vars=value_cols,
                    value_name='EuroToCurrency')
    hist['CurrencyToEuro']=1/hist['EuroToCurrency']
    hist['Date']=pd.to_datetime(hist['Date'], utc=True)
    hist['CurrencyCode']=hist['CurrencyCode'].str.strip()
    return hist


In [57]:
indata['Date'].max()>max_hist_date

False

## Seed the table

In [41]:

def data_is_new(indata):
    indata=load_data('new')
    indata=process_data(indata)
    indata['Date'].max()
    max_hist_date=pandas_gbq.read_gbq("""
            SELECT max(Date) FROM `ds-smartsupply.currency_exchange.euro_rates_daily`
            """).values.item()
    return indata['Date'].max()>max_hist_date

def upload_to_bq(hist_bq:pd.DataFrame, if_exists:str='replace'):
    project_id = 'whatever'
    dataset_id = 'currency_exchange'
    table_id = 'euro_rates_daily'
    pandas_gbq.to_gbq(hist_bq,  f'{dataset_id}.{table_id}', project_id=project_id, if_exists=if_exists, location='europe-west2')
    message_string = f'{len(hist_bq)} rows uploaded to table `{dataset_id}.{table_id}`'
    print(message_string)
    return({'status':message_string})


def seed_historical():
    try:
        hist = load_data('hist')
        hist = process_data(hist)
        upload_to_bq(hist)
        return 'success'
    except: return 'failure'

def append_new():
    try:
        new = load_data('new')
        new = process_data(new)
        if data_is_new(indata=new):
            upload_to_bq(new, if_exists='append')
        else:
            print('no new data to append')
        return 'success'
    except:
        return 'failure'

